## Librerías y acceso

In [1]:
import pandas as pd
import requests
import json
import datetime
import finnhub
import yfinance as yf
import numpy as np
import random

FH_token = ''
finnhub_client = finnhub.Client(api_key=FH_token)

## Tickers y sectores

Voy a quedarme con un único sector, y tomar 10 activos para comparar rendimientos con el de ese promedio. Buscamos activos que les vaya mejor que otros similares por sus fundamentos como empresa.

In [2]:
import requests
from bs4 import BeautifulSoup
table = BeautifulSoup(requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies').text, 'lxml').find('table', {'class': 'wikitable sortable'})
tickers = []
sectors = []
for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        sector = row.findAll('td')[3].text
        ticker = ticker[:-1]
        sectors.append(sector)
        tickers.append(ticker)
tickers_sp500 = tickers
tickers = pd.DataFrame({'Ticker': tickers, 'Sector': sectors})
tickers_sectors_sample10 = list(tickers[tickers.Sector == random.sample(sectors, 1)[0]].sample(10)['Ticker'])

# Descarga data de balances

In [3]:
full_data_balances = pd.DataFrame()
for t in tickers_sectors_sample10:
    print(t)
    js = finnhub_client.company_basic_financials(t, 'all')
    full_data_ticker = pd.DataFrame()
    try:
        series = js['series']['quarterly']
        for i in series.keys():
            data_serie = pd.DataFrame(series[i])
            if len(data_serie) == 0:
                pass
            else:
                data_serie.columns = ['fecha_fin_cuatrimestre_fiscal', i]
                if len(full_data_ticker) == 0:
                    full_data_ticker = data_serie.copy()
                else:
                    full_data_ticker = pd.merge(full_data_ticker, data_serie, how='outer', on=['fecha_fin_cuatrimestre_fiscal'])
        full_data_ticker.insert(0,'ticker',t)
        full_data_balances = pd.concat([full_data_balances, full_data_ticker])
    except:
        pass
full_data_balances = full_data_balances.sort_values(by=['ticker', 'fecha_fin_cuatrimestre_fiscal']).reset_index(drop=True)
full_data_balances.insert(0,'fecha_aproximada_presentacion',pd.to_datetime(full_data_balances['fecha_fin_cuatrimestre_fiscal']) + datetime.timedelta(days=45))

ETN
EXPD
CMI
LDOS
PNR
HON
GE
GD
IEX
SWK


## Descarga de data de precios y add target

In [4]:
full_data = full_data_balances.copy()
tickers_efectivos = list(full_data.ticker.unique())
df = yf.download(tickers_efectivos, start=full_data.fecha_aproximada_presentacion.min(), end=full_data.fecha_aproximada_presentacion.max()).droplevel(level=0, axis=1)
df = df.iloc[:, 0:len(tickers_efectivos)].pct_change()
df['BENCHMARK'] = df.iloc[:, 0:len(tickers_efectivos)].median(axis=1)
df_backup = df.copy()
for col in df.iloc[:, 0:len(tickers_efectivos)].columns:
    df[col] = df[col] - df['BENCHMARK']
    
# Target
full_data['fecha_aproximada_siguiente_presentacion'] = full_data.fecha_aproximada_presentacion.shift(-1)
full_data = full_data[full_data['fecha_aproximada_siguiente_presentacion'] > full_data['fecha_aproximada_presentacion']].reset_index(drop=True)
serie_fi = full_data.fecha_aproximada_presentacion.values
serie_ff = full_data.fecha_aproximada_siguiente_presentacion.values
serie_t = full_data.ticker.values
targets = []
for fi, ff, t in zip(serie_fi, serie_ff, serie_t):
    ultima_fila = ((df.loc[fi:ff, t]+1).cumprod() - 1)
    try:
        targets.append(ultima_fila[-1])
    except:
        targets.append(np.nan)
        
full_data['target'] = targets

[*********************100%***********************]  10 of 10 completed


## Análisis de relaciones

### Relación de la variable nominal con el target

In [5]:
print(full_data.corrwith(full_data['target']))

bookValue                  -0.103098
cashRatio                  -0.006486
currentRatio                0.064705
ebitPerShare               -0.075917
eps                        -0.077934
ev                         -0.106464
fcfMargin                  -0.040478
fcfPerShareTTM             -0.016551
grossMargin                -0.045623
longtermDebtTotalAsset     -0.043411
longtermDebtTotalCapital   -0.035159
longtermDebtTotalEquity    -0.067729
netDebtToTotalCapital      -0.052686
netDebtToTotalEquity       -0.082596
netMargin                  -0.111250
operatingMargin            -0.104093
pb                         -0.049127
peTTM                      -0.044098
pfcfTTM                    -0.039340
pretaxMargin               -0.093776
psTTM                      -0.096117
quickRatio                  0.050570
roaTTM                      0.041746
roeTTM                      0.013319
roicTTM                     0.053663
rotcTTM                     0.059045
salesPerShare              -0.031555
s

### Relación de la variación de la variable con respecto al período pasado con el target

In [6]:
full_data_variations = full_data.iloc[:, 3:-2].pct_change()
full_data_variations['ticker'] = full_data['ticker']
full_data_variations['target'] = full_data['target']
full_data_variations[full_data_variations.ticker == full_data_variations.ticker.shift(1)]
print(full_data_variations.corrwith(full_data_variations['target']))

bookValue                   0.010849
cashRatio                   0.044392
currentRatio               -0.037425
ebitPerShare               -0.024988
eps                        -0.050122
ev                          0.052002
fcfMargin                   0.051232
fcfPerShareTTM              0.041841
grossMargin                -0.014555
longtermDebtTotalAsset     -0.002236
longtermDebtTotalCapital    0.014061
longtermDebtTotalEquity     0.029007
netDebtToTotalCapital       0.031217
netDebtToTotalEquity        0.037077
netMargin                  -0.048083
operatingMargin            -0.020389
pb                          0.023333
peTTM                       0.019783
pfcfTTM                    -0.021151
pretaxMargin                0.007061
psTTM                      -0.005119
quickRatio                  0.005601
roaTTM                     -0.040028
roeTTM                     -0.047248
roicTTM                    -0.038341
rotcTTM                    -0.028983
salesPerShare              -0.010916
s